In [285]:
import numpy as np
import pandas as pd
import re


from nltk.corpus import stopwords
from string import ascii_lowercase
import pandas as pd
import gensim, os, re, itertools, nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from keras.preprocessing import text, sequence

#### Load all data

In [317]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

#### Extract IP information

And then split them into four partitions.

In [261]:
def extractIP(line):
    ip = re.findall(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b", line)
    if ip != []:
        return ip[0]
    else :
        return '0.0.0.0'
    
def splitIP(x):
    return x.split('.')[t]

In [262]:
t = 0
train['ip_0']=train['comment_text'].apply(extractIP).apply(splitIP)
test['ip_0']=test['comment_text'].apply(extractIP).apply(splitIP)
t = 1
train['ip_1']=train['comment_text'].apply(extractIP).apply(splitIP)
test['ip_1']=test['comment_text'].apply(extractIP).apply(splitIP)
t = 2
train['ip_2']=train['comment_text'].apply(extractIP).apply(splitIP)
test['ip_2']=test['comment_text'].apply(extractIP).apply(splitIP)
t = 3
train['ip_3']=train['comment_text'].apply(extractIP).apply(splitIP)
test['ip_3']=test['comment_text'].apply(extractIP).apply(splitIP)

#### Text Cleaning

Mainly use keras text to deal with this problem. Save all necessary punctuation.

In [318]:
train["comment_text"].fillna("fillna")
test["comment_text"].fillna("fillna")
X_train = train["comment_text"].str.lower()
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

X_test = test["comment_text"].str.lower()

In [288]:
max_features=1000
maxlen=150
embed_size=300

In [289]:
tok=text.Tokenizer(num_words=max_features,lower=True)
tok.fit_on_texts(list(X_train)+list(X_test))
X_train=tok.texts_to_sequences(X_train)
X_test=tok.texts_to_sequences(X_test)
x_train_seq=sequence.pad_sequences(X_train,maxlen=maxlen)
x_test_seq=sequence.pad_sequences(X_test,maxlen=maxlen)

In [301]:
np.save('x_test_seq.npy',x_test_seq)
np.save('x_train_seq.npy',x_train_seq)

### Load data Here

Output all data with matrix

In [271]:
train = pd.read_csv('../output/train_pre.csv')
test = pd.read_csv('../output/test_pre.csv')

In [280]:
EMBEDDING_FILE = '../input/crawl-300d-2M.vec'
EMBEDDING_FILE_Glove='../input/glove.840B.300d.txt'

In [282]:
embeddings_index = {}
with open(EMBEDDING_FILE_Glove,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [283]:
word_index = tok.word_index
#prepare embedding matrix
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
np.save('../output/embedMatrix.npy',embedding_matrix)

In [284]:
np.save('../output/embedMatrixglove.npy',embedding_matrix)

In [303]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

#### add new fetures|

In [266]:
def addNewFeature(df):
    df['total_length'] = df['comment_text'].apply(len)
    df['capitals'] = df['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    df['caps_vs_length'] = df.apply(lambda row: float(row['capitals'])/float(row['total_length']),
                                    axis=1)
    df['num_punctuation'] = df['comment_text'].apply(
        lambda comment: sum(comment.count(w) for w in '.,;:'))
    df['num_unique_words'] = df['comment_text'].apply(lambda comment: len(set(w for w in comment.split())))
    df['num_exclamation_marks'] = df['comment_text'].apply(lambda comment: comment.count('!'))

In [278]:
addNewFeature(train)
addNewFeature(test)

#### TDIDF

#### Topic LDM

In [339]:
doc = list(X_train)+list(X_test)

In [341]:
from gensim import corpora
from collections import defaultdict
from nltk.corpus import stopwords

documents = doc
stoplist = set(stopwords.words('english'))
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# 去掉只出现一次的单词
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

dictionary = corpora.Dictionary(texts)   # 生成词典
corpus = [dictionary.doc2bow(text) for text in texts]

In [343]:
from gensim import corpora, models, similarities
tfidf_model = models.TfidfModel(corpus) 
corpus_tfidf = tfidf_model[corpus] ## all tfidf

In [344]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=100)
corpus_lsi = lsi_model[corpus_tfidf]

In [345]:
lda_model = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=100)
corpus_lda = lda_model[corpus_tfidf]

/usr/local/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [346]:
lsi_model.save('model.lsi') # same for tfidf, lda, ...
lda_model.save('model.lda')
tfidf_model.save('model.tfidf')

In [372]:
stoplist = set(stopwords.words('english'))
traintext = [[word for word in document.lower().split() if word not in stoplist]
         for document in list(X_train)]

# 去掉只出现一次的单词
frequency = defaultdict(int)
for text in traintext:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

In [373]:
stoplist = set(stopwords.words('english'))
testtext = [[word for word in document.lower().split() if word not in stoplist]
         for document in list(X_test)]

# 去掉只出现一次的单词
frequency = defaultdict(int)
for text in testtext:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

In [374]:
testtext_corpus = [dictionary.doc2bow(text) for text in testtext]
traintext_corpus = [dictionary.doc2bow(text) for text in traintext]

In [375]:
test_tfidf = tfidf_model[testtext_corpus]
test_lsi = lsi_model[testtext_corpus]
test_lda = lda_model[testtext_corpus]

In [376]:
train_tfidf = tfidf_model[traintext_corpus]
train_lsi = lsi_model[traintext_corpus]
train_lda = lda_model[traintext_corpus]

In [409]:
lda_model.get_document_topics(traintext_corpus)

In [410]:
t = train_lsi.corpus

In [413]:
t = similarities.MatrixSimilarity(train_lsi)

In [ ]:
sims = t[train_lsi]

In [1]:
train_tfidf

NameError: name 'train_tfidf' is not defined